In [95]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

In [96]:
db = database('trfc.db')

In [97]:
app, rt = fast_app()

In [98]:
user_inputs = {
    'min_season': 1921,
    'max_season': 2024,
    'league_tiers': [2, 3, 4, 5],
    'inc_play_offs': 1,
    'generic_comps': ['Anglo-Italian Cup', "Associate Members' Cup", 'FA Cup', 'FA Trophy', "Full Members' Cup", 'League Cup'],
    'pens_as_draw': 1,
    'venues': ['H', 'A', 'N'],
    'min_games': 10,
    'inc_caretaker': 1,
}

In [99]:
min_season = user_inputs['min_season']
max_season = user_inputs['max_season']
league_tiers = user_inputs['league_tiers']
inc_play_offs = user_inputs['inc_play_offs']
pens_as_draw = user_inputs['pens_as_draw']
venues = user_inputs['venues']
min_games = user_inputs['min_games']
inc_caretaker = user_inputs['inc_caretaker']

In [100]:
if inc_play_offs == 0:
    po_filter = 'AND COALESCE(c.is_playoff, 0) != 1'
else:
    po_filter = ''

In [101]:
venue_placeholders = ','.join(['?' for _ in venues])

In [102]:
tier_placeholders = ','.join(['?' for _ in user_inputs['league_tiers']]) if user_inputs['league_tiers'] else ''

comp_placeholders = ','.join(['?' for _ in user_inputs['generic_comps']]) if user_inputs['generic_comps'] else ''

In [103]:
tier_comp_filter = ''
if tier_placeholders or comp_placeholders:
    filters = []
    if tier_placeholders:
        filters.append(f'r.league_tier IN ({tier_placeholders})')
    if comp_placeholders:
        filters.append(f'r.generic_comp IN ({comp_placeholders})')
    tier_comp_filter = 'AND (' + ' OR '.join(filters) + ')'

In [104]:
if inc_caretaker == 0:
    caretaker_filter = 'AND mr.mgr_role != "Caretaker"'
else:
    caretaker_filter = ''

In [105]:
pens_as_draw = 1

In [106]:
query = f'''
    SELECT
        m.manager_name,
        COUNT(*) as P,
        COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) = 0 AND r.outcome = 'D' AND c.pens_outcome = 'W') OR r.outcome = 'W'))
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) as W,
        COUNT(
            CASE WHEN
                (? = 0 AND r.outcome = 'D' AND (COALESCE(c.is_pen_shootout, 0) = 0 OR (COALESCE(c.is_pen_shootout, 0) = 1) AND COALESCE(c.is_multi_leg, 0) = 1))
            OR 
                (? = 1 AND r.outcome = 'D')
            THEN 1 END) as D,
        COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) = 0 AND r.outcome = 'D' AND c.pens_outcome = 'L') OR r.outcome = 'L'))
            OR 
                (? = 1 AND r.outcome = 'L')
            THEN 1 END) as L,
        SUM(r.goals_for) as GF,
        SUM(r.goals_against) as GA,
        SUM(r.goals_for) - SUM(r.goals_against) as GD,
        ROUND(CAST(COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) != 1 AND r.outcome = 'D' AND c.pens_outcome = 'W')) OR r.outcome = 'W')
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) AS FLOAT) / COUNT(*) * 100, 1) as win_pc
    FROM results r
    LEFT JOIN cup_game_details c ON r.game_date = c.game_date
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN seasons s ON r.season = s.season
    WHERE s.ssn_start >= ?
        AND s.ssn_start <= ?
        AND r.venue IN ({venue_placeholders})
        {po_filter}
        {tier_comp_filter}
        {caretaker_filter}
    GROUP BY m.manager_name
    HAVING COUNT(*) >= ?
    ORDER BY P DESC
'''

params = [
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    min_season, max_season,
    *venues
]

if user_inputs['league_tiers']:
    params.extend(user_inputs['league_tiers'])

if user_inputs['generic_comps']:
    params.extend(user_inputs['generic_comps'])

params.append(min_games)

results = db.execute(query, tuple(params))

df = pd.DataFrame(
    results.fetchall(),
    columns=[d[0] for d in results.description]
)

df.query("manager_name=='Nigel Adkins'")

,manager_name,P,W,D,L,GF,GA,GD,win_pc
16,Nigel Adkins,69,24,13,32,88.0,105.0,-17.0,34.8


In [107]:
pens_as_draw = 0

In [108]:
query = f'''
    SELECT
        manager_name,
        COUNT(*) as P,
        COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) = 0 AND r.outcome = 'D' AND c.pens_outcome = 'W') OR r.outcome = 'W'))
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) as W,
        COUNT(
            CASE WHEN
                (? = 0 AND r.outcome = 'D' AND (COALESCE(c.is_pen_shootout, 0) = 0 OR (COALESCE(c.is_pen_shootout, 0) = 1) AND COALESCE(c.is_multi_leg, 0) = 1))
            OR 
                (? = 1 AND r.outcome = 'D')
            THEN 1 END) as D,
        COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) = 0 AND r.outcome = 'D' AND c.pens_outcome = 'L') OR r.outcome = 'L'))
            OR 
                (? = 1 AND r.outcome = 'L')
            THEN 1 END) as L,
        SUM(r.goals_for) as GF,
        SUM(r.goals_against) as GA,
        SUM(r.goals_for) - SUM(r.goals_against) as GD,
        ROUND(CAST(COUNT(
            CASE WHEN
                (? = 0 AND ((COALESCE(c.is_multi_leg, 0) != 1 AND r.outcome = 'D' AND c.pens_outcome = 'W')) OR r.outcome = 'W')
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) AS FLOAT) / COUNT(*) * 100, 1) as win_pc
    FROM results r
    LEFT JOIN cup_game_details c ON r.game_date = c.game_date
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN seasons s ON r.season = s.season
    WHERE s.ssn_start >= ?
        AND s.ssn_start <= ?
        AND r.venue IN ({venue_placeholders})
        {po_filter}
        {tier_comp_filter}
        {caretaker_filter}
    GROUP BY manager_name
    HAVING COUNT(*) >= ?
    ORDER BY P DESC
'''

params = [
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    min_season, max_season,
    *venues
]

if user_inputs['league_tiers']:
    params.extend(user_inputs['league_tiers'])

if user_inputs['generic_comps']:
    params.extend(user_inputs['generic_comps'])

params.append(min_games)

results = db.execute(query, tuple(params))

df = pd.DataFrame(
    results.fetchall(),
    columns=[d[0] for d in results.description]
)

df.query("manager_name=='Nigel Adkins'")

,manager_name,P,W,D,L,GF,GA,GD,win_pc
16,Nigel Adkins,69,24,12,33,88.0,105.0,-17.0,34.8
